# **CSE-601 Project-3 K-NearestNeighbor**



In [1]:
import numpy as np
import math
import pandas as pd
import sys
from random import randrange


In [2]:
def KNN(X_train,X_test,Y_train,k):

    tx1 = X_train.copy()
    tx2 = Y_train.copy()
    
    tx1['actualvalue'] = tx2 

    finalans = []
    for index,row in X_test.iterrows():
        row = row.values.reshape(1,row.shape[0])
        t = np.tile(row,(X_train.shape[0],1))
        x1 = np.sqrt(np.sum((X_train - t)**2,1))
        x1 = x1.to_frame()
        tx1['Edistance'] = x1
        adddist1 = tx1.sort_values('Edistance')
        ndata = adddist1.iloc[0:k]
        ndata2 = ndata.groupby('actualvalue')['actualvalue'].count()
        ndata2 = ndata2.sort_values(ascending= False)
        test =  ndata2.keys()
        ans = test[0]
        finalans.append(ans)
    return finalans

In [3]:
def metrics(actual, predicted):

    tp = fn = fp= tn = 0
    accuracy = precision = recall = f1_measure = 0

    for i in range(len(predicted)):
        if actual[i] == 1 and predicted[i] == 1:
            tp += 1
        elif actual[i] == 1 and predicted[i] == 0:
            fn += 1
        elif actual[i] == 0 and predicted[i] == 1:
            fp += 1
        elif actual[i] == 0 and predicted[i] == 0:
            tn += 1

    accuracy += (float(tp+tn)/(tp+fn+fp+tn))

    if(tp+fp != 0):
        precision += (float(tp)/(tp+fp))

    if(tp+fn != 0):
        recall += (float(tp)/(tp+fn))

    f1_measure += (float(2*tp)/((2*tp)+fn+fp))

    return accuracy, precision, recall, f1_measure

In [4]:
def cross_validation_split(dataset, folds=3):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / folds)
        for i in range(folds):
            fold = list()
            while len(fold) < fold_size:
                index = randrange(len(dataset_copy))
                fold.append(dataset_copy.pop(index))
            dataset_split.append(fold)
        return np.asarray(dataset_split)

In [5]:
def preprocess(inputfile):
    with open(inputfile) as textFile:
        lines=[line.split("\t") for line in textFile]
    data=np.asarray(lines)
    
    #Handling Categorical data
    sample_data = data[1]
    categoricaldata_cols=[]
    for col in range(data.shape[1]):
        feature = sample_data[col]
        if feature.isalpha():
            array,i = np.unique(data[:,col],return_inverse = True)
            data[:,col] = i.astype(np.float)
            categoricaldata_cols.append(col) 
            
    points= np.matrix(data[:,:-1],dtype=float,copy=False)
    ground_truth_labels = np.asarray(data[:,-1],dtype=int)
            
    #Normalising the data
    for col in range(points.shape[1]):
        if col not in categoricaldata_cols:
            v = points[:,col]
            points[:,col] = (v - v.min()) / (v.max() - v.min())

    return points,ground_truth_labels

In [13]:
file_name = input("enter the file name :")

k = int(input("enter value of k in K-fold cross validation:"))
k1 = int(input("enter value of k in K-NN :"))

points,ground_truth_labels=preprocess(file_name)

df = pd.DataFrame(points)
df[df.shape[1]] = ground_truth_labels
dataset_split  =  cross_validation_split(df.to_numpy(), 10)
dataset_split = dataset_split.tolist()

res = []
acc_sum = 0 
prec_sum = 0
recall_sum = 0
f1_measure_sum = 0 

for i in range(len(dataset_split)):
    train = dataset_split[:i] + dataset_split[i+1:]
    test = dataset_split[i]
    test =  pd.DataFrame(test)
    df_train =  pd.DataFrame()
    for j in range(len(train)):
        t =  pd.DataFrame(train[j])
        df_train = df_train.append(t)
    test_label=KNN(df_train.iloc[:,:-1],test.iloc[:,:-1],df_train.iloc[:,-1:],1)
    acc, prec, recall, f1_measure = metrics(test_label,test[test.columns[-1]])
    acc_sum = acc_sum+acc
    prec_sum = prec_sum+prec
    recall_sum =recall_sum+recall
    f1_measure_sum = f1_measure_sum+f1_measure
print("the mean values of metrics are :")
print("Accuracy",(acc_sum/(k))*100)
print("Precision",(prec_sum/(k))*100)
print("Recall",(recall_sum/(k))*100)
print("F1-Measure",f1_measure_sum/(k))

enter the file name :project3_dataset2.txt
enter value of k in K-fold cross validation:10
enter value of k in K-NN :10
the mean values of metrics are :
Accuracy 66.30434782608697
Precision 46.06903145873734
Recall 51.999376354639516
F1-Measure 0.4836826271197795
